In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import sys
import time
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import time
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
import matplotlib.pyplot as plt

import random , re

%matplotlib inline

Using TensorFlow backend.


In [0]:
if torch.cuda.is_available():      
	device = torch.device("cuda")
	print('There are %d GPU(s) available.' % torch.cuda.device_count())
	print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
	print('No GPU available, using the CPU instead.')
	device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [0]:
# device = 'cpu'

In [0]:
seed_val = 45
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
batchsize = 64

In [0]:
basepath = "./drive/My Drive/AI Ethics Term Project/"
path = basepath

In [0]:
class TextCNN(nn.Module):
    
    def __init__(self, V, D=100 ,C=1 ,Co= 32,Ks = [7,7,3,3,3],dropout=0.5):
        super(TextCNN, self).__init__()
        # self.args = args
        
        self.V = V
        self.D = D
        self.C = C
        self.Ci = 1
        self.Co = Co
        self.Ks = Ks
        self.dropout = dropout

        self.embed = nn.Embedding(self.V, self.D)
        # self.word_embeddings.weight = nn.Parameter(weights, requires_grad=False)
        # self.convs1 = [nn.Conv2d(Ci, Co, (K, D)) for K in Ks]
        self.convs1 = nn.ModuleList([nn.Conv2d(self.Ci, self.Co, (K, self.D)) for K in self.Ks])
        '''
        self.conv13 = nn.Conv2d(Ci, Co, (3, D))
        self.conv14 = nn.Conv2d(Ci, Co, (4, D))
        self.conv15 = nn.Conv2d(Ci, Co, (5, D))
        '''
        self.dropout = nn.Dropout(self.dropout)
        self.fc1 = nn.Linear(len(self.Ks)*self.Co, self.C)

    def conv_and_pool(self, x, conv):
        x = F.relu(conv(x)).squeeze(3)  # (N, Co, W)
        x = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x

    def forward(self, x):
        x = self.embed(x)  # (N, W, D)
        
        # if self.args.static:
            # x = Variable(x)

        x = x.unsqueeze(1)  # (N, Ci, W, D)

        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, W), ...]*len(Ks)

        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)

        x = torch.cat(x, 1)

        '''
        x1 = self.conv_and_pool(x,self.conv13) #(N,Co)
        x2 = self.conv_and_pool(x,self.conv14) #(N,Co)
        x3 = self.conv_and_pool(x,self.conv15) #(N,Co)
        x = torch.cat((x1, x2, x3), 1) # (N,len(Ks)*Co)
        '''
        x = self.dropout(x)  # (N, len(Ks)*Co)
        logit = self.fc1(x)  # (N, C)
        return logit

In [0]:

train_id_terms = ['lgbt',
 'taoist',
 'american',
 'latina',
 'asian',
 'millenial',
 'buddhist',
 'hispanic',
 'lesbian',
 'canadian',
 'indian',
 'heterosexual',
 'christian',
 'male',
 'younger',
 'catholic',
 'latino',
 'latinx',
 'transgender',
 'muslim',
 'female',
 'chinese',
 'sikh',
 'deaf',
 'paralyzed',
 'homosexual',
 'older',
 'young',
 'jewish',
 'straight',
 'protestant',
 'old',
 'bisexual',
 'elderly',
 'trans',
 ]
 

test_id_terms = ['blind',
                 'nonbinary',
                 'queer',
                 'gay',
                 'lgbtq',
                 'mexican',
                 'european',
                 'african',
                 'teenage',
                 'black',
                 'white',
                 'japanese',
                 'african american',
                 'middle eastern',
                 'middle aged']



In [0]:
def load_data(mode='train'):
    if mode=='train':
        train_df = pd.read_csv(basepath+'cleaned_train.csv')
        # train_df = shuffle(train_df)
        train_df = train_df.set_index(np.random.permutation(train_df.index))
        train_df.reset_index(inplace=True, drop=True)
        # train_df = train_df
        train_len = int(0.8*train_df.shape[0])
        val_df = train_df[train_len:]
        train_df = train_df[:train_len]
        return list(train_df['comment_text'].values),list(train_df['toxic'].values),list(val_df['comment_text'].values),list(val_df['toxic'])
    if mode=='test':
        test_df = pd.read_csv(basepath+'test_cleaned.csv')
        return list(test_df['comment_text'].values),list(test_df['toxic'].values)

In [0]:
test_df = pd.read_csv(path+'test_cleaned.csv')
train_df = pd.read_csv(basepath+'cleaned_train.csv')

In [0]:
texts = []
train_texts,train_y,val_texts,val_y = load_data('train')
test_texts,test_y = load_data('test')
tokenizer = Tokenizer(num_words=5000, oov_token=True) #


In [0]:
tokenizer.fit_on_texts(train_texts)

In [0]:
tokenizer.word_index[tokenizer.oov_token]

1

In [0]:
tokenizer.word_index = {e:i for e,i in tokenizer.word_index.items() if i <= 5000}

count=0
for word in train_id_terms+test_id_terms:
  if not tokenizer.word_index.get(word) or tokenizer.word_index.get(word)>5000:
    count+=1
print(count)

tokenizer.word_index = {e:i for e,i in tokenizer.word_index.items() if i <= 5000-count} # <= because tokenizer is 1 indexed

for word in train_id_terms+test_id_terms:
  if not tokenizer.word_index.get(word):
    tokenizer.word_index[word] = 5000-count+1
    count-=1

23


In [0]:
train_tokens = tokenizer.texts_to_sequences(train_id_terms)
test_tokens = tokenizer.texts_to_sequences(test_id_terms)
train_tokens = [i[0] for i in train_tokens]
test_tokens = [i[0] for i in test_tokens]


In [0]:
# print(train_tokens)

In [0]:
# # tokenizer.word_index[tokenizer.oov_token]
# for k,v in tokenizer.word_index.items():
#   if v>=5000-24:
#     print(k,v)

In [0]:
# for word in train_id_terms+test_id_terms:
#   print(word, tokenizer.word_index.get(word))

In [0]:
def train_dataloader():
    for i in range(0, len(train_data), batchsize):
        yield torch.tensor(train_data[i:i+batchsize], device=device, dtype=torch.long),torch.tensor(train_y[i:i+batchsize],device=device)

In [0]:
def val_dataloader():
    for i in range(0, len(val_data), batchsize):
        yield torch.tensor(val_data[i:i+batchsize], device=device, dtype=torch.long),torch.tensor(val_y[i:i+batchsize],device=device)

In [0]:
def test_dataloader():
  for i in range(0,len(test_data),batchsize):
    yield torch.tensor(test_data[i:i+batchsize],device=device,dtype=torch.long),torch.tensor(test_y[i:i+batchsize],device=device)

In [0]:
print(len(tokenizer.word_index))
print(tokenizer.num_words)

5000
5000


In [0]:
# # tokenizer.word_index.get('racist')

# train_toxic = train_df[train_df['toxic']==1]
# train_toxic_texts = list(train_toxic['comment_text'].values)

# tk_temp = Tokenizer(num_words=1000, lower=True)
# tk_temp.fit_on_texts(train_toxic_texts)

# w =  sorted(tk_temp.word_counts.items(), key=lambda x:x[1], reverse=True)
# # print(len(w))

# count=1
# temp = []
# for k in w:
#   if not tokenizer.word_index.get(k[0]):
#     temp.append(k[0])
#     # print(k[0], k[1])
#     tokenizer.word_index[k[0]] = 5000+count
#     count +=1 
#   if count==1001:
#     break

# # print(temp)
# print('No of added words ', len(temp))

In [0]:
# print(tokenizer.num_words)
# tokenizer.word_index.get('trump')

In [0]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [0]:
train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)
train_data = pad_sequences(train_sequences, maxlen=100, padding='post')
val_data = pad_sequences(val_sequences,maxlen=100,padding='post')
test_data = pad_sequences(test_sequences, maxlen=100, padding='post')

In [0]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [0]:
def save_model(model, name, val_loss=0):
  state = {
      'model':model.state_dict(),
      'optimizer': optimizer.state_dict(),
      'val_loss': val_loss
  }
  torch.save(state, path+name)

def load_model(model, name):
  state = torch.load(path+name)
  model.load_state_dict(state['model'])
  optimizer.load_state_dict(state['optimizer'])
  print('Validation loss of the model is ', state.get('val_loss'))
  return state.get('val_loss')

In [0]:
def generate_cfs(x):
    cf_x = []
    cx = x.detach().cpu()
    tokens= train_tokens.copy()
    for i in cx:
        # indices = torch.ones_like(i, dtype = torch.uint8, device =device)
        i = i.tolist()
        ids = np.intersect1d(i, tokens).tolist()
        if len(ids)>0:
            ind = i.index(ids[0])
            tokens.remove(ids[0])
            i[ind] = random.choice(tokens)
        cf_x.append(i)
    return torch.tensor(cf_x,device=device,dtype=torch.long)

In [0]:
def compute_clp_loss(x):
  fx = textcnn(x)
  f_cfx = textcnn(generate_cfs(x))
  loss = torch.sum(abs(fx-f_cfx), dim=0)/x.shape[0]
  return loss

In [0]:
def compute_clp_nontoxic_loss(x, y):
  y_invert = y
  y_invert[y_invert==0]=2
  y_invert[y_invert==1]=0
  y_invert[y_invert==2]=1
  fx = (textcnn(x))
  f_cfx = (textcnn(generate_cfs(x)))
  # print(abs(fx-f_cfx).shape , y_invert.shape)
  loss = torch.sum(abs(fx-f_cfx).squeeze(1) * y_invert, dim=0)/x.shape[0]
  # print(loss.shape)
  return loss

In [0]:
print(len(train_y))
print(sum(train_y))
pw = (len(train_y) - sum(train_y))/sum(train_y)
print(pw)

127656
12257
9.414946561148732


In [0]:
# class_wt = torch.tensor(compute_class_weight('balanced',np.unique(train_y),train_y),device=device,dtype=torch.float32)
textcnn = TextCNN(V = (tokenizer.num_words))
if torch.cuda.is_available():
  textcnn.cuda()

class_wt = torch.tensor(compute_class_weight('balanced',np.unique(train_y),train_y),device=device,dtype=torch.float)

# print(class_wt)
pw = class_wt[1]/class_wt[0]
print(pw)

criterion = nn.BCEWithLogitsLoss(pos_weight = pw) # with sigmoid # pos_weight = pw
# clp_loss_criterion = nn.L1Loss(reduction='mean')
optimizer = torch.optim.Adam(textcnn.parameters(),lr =1e-3)

tensor(9.4149, device='cuda:0')


In [0]:
softmax = torch.nn.Softmax(dim=1)
sigmoid= torch.nn.Sigmoid()

In [0]:
# load_model(textcnn)

In [0]:
print('Total number of trainable parameters: ', sum(p.numel() for p in textcnn.parameters() if p.requires_grad)/float(1000000), 'M')

Total number of trainable parameters:  0.573921 M


In [0]:
use_clp = False
use_clp_nontoxic= True # only lambda 1 for this
lambda_clp =1  # 0.05,1,5
name = 'textcnn_clpnontoxic_lam1.pt'
min_val_loss = float('INF')

In [0]:
# min_val_loss = load_model(textcnn, name)
# print(min_val_loss)

In [0]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.96)

In [0]:
# a = torch.tensor([0.56, 0.54, 0.22])
# (a>=0.5).long()

In [0]:
true_y = []
pred_y = []
state = {}

for epoch in range(15):
    print('======== Epoch {:} / {:} ========'.format(epoch+1,15))
    t0 = time.time()
    textcnn.train()
    total_loss = 0
    total_steps = 0
    pred_logits = []
    one_hot_y = []
    true_labels = []
    pred_labels = []
    pred_logits = []

    for step,batch in (enumerate(train_dataloader())):

        total_steps+=1
        # if step % 40 == 0 and not step == 0:
        #         # Calculate elapsed time in minutes.
        #         elapsed = format_time(time.time() - t0)
        x,y = batch
        cf_x = generate_cfs(x)
        textcnn.zero_grad()
        fx = (textcnn(x)).squeeze(1)

    
        if use_clp:
          loss = criterion(fx,y.float()) + lambda_clp * compute_clp_loss(x)
          # print(criterion(fx,y.float()).item()  , lambda_clp * compute_clp_loss(x)  )
        elif use_clp_nontoxic:
          loss = criterion(fx,y.float()) + lambda_clp * compute_clp_nontoxic_loss(x, y)
          # print( criterion(fx,y.float()).shape,   (lambda_clp * compute_clp_nontoxic_loss(x, y)).shape )
        else:
          loss = criterion(fx,y.float())

        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(textcnn.parameters(), 0.7)
        optimizer.step()
        
        true_label_vals = y.to('cpu').numpy()
        true_labels += list(true_label_vals.flatten())

        pred_labels += list((fx>=0.5).long().detach().cpu().numpy())
        pred_logits.extend(fx.detach().cpu().numpy())
        # one_hot_y.extend(np.eye(2)[true_label_vals])
        
    avg_train_loss = total_loss / total_steps

    matrix = confusion_matrix(true_labels, pred_labels)
    class_acc = matrix.diagonal()/matrix.sum(axis=1)

    print('Training loss: ',avg_train_loss)
    print('Accuracy: ',accuracy_score(true_labels,pred_labels))
    print('Macro F1 score: ',f1_score(true_labels,pred_labels,average='macro'))
    print('ROC_AUC Score: ',roc_auc_score(true_labels , pred_logits ))
    print(matrix)
    print('Class accuracies: ', class_acc)
    #############################test data #####################################
    textcnn.eval()
    pred_logits = []
    true_labels = []
    pred_labels = []
    pred_scores = []
    with torch.no_grad():
      for x,y in test_dataloader():
        pred_y = textcnn(x)
        fx = (pred_y).squeeze(1)
        fx = sigmoid(fx)

        true_label_vals = y.to('cpu').numpy()
        true_labels += list(true_label_vals.flatten())

        pred_labels += list((fx>=0.5).long().detach().cpu().numpy())
        pred_logits.extend(fx.detach().cpu().numpy())
        # one_hot_y.extend(np.eye(2)[true_label_vals])

    matrix = confusion_matrix(true_labels, pred_labels)
    class_acc = matrix.diagonal()/matrix.sum(axis=1)
    print('Test Accuracy: ',accuracy_score(true_labels,pred_labels))
    print('Test Macro F1 score: ',f1_score(true_labels,pred_labels,average='macro'))
    print('Test ROC AUC Score: ',roc_auc_score(true_labels,  pred_logits  ))
    print(matrix)
    print('Class accuracies: ', class_acc)
    ###################################### val data ###############################################
    true_labels = []
    pred_labels = []
    pred_scores = []
    pred_logits = []
    val_loss = 0
    j = 0
    with torch.no_grad():
      for x,y in val_dataloader():
        pred_y = textcnn(x)
        fx = (pred_y).squeeze(1)

        if use_clp:
          loss = criterion(fx,y.float()) + lambda_clp * compute_clp_loss(x)
        elif use_clp_nontoxic:
          loss = criterion(fx,y.float()) + lambda_clp * compute_clp_nontoxic_loss(x, y)
        else:
          loss = criterion(fx,y.float())

        fx = sigmoid(fx)
        true_label_vals = y.to('cpu').numpy()
        true_labels += list(true_label_vals.flatten())

        pred_labels += list((fx>=0.5).long().detach().cpu().numpy())
        pred_logits.extend(fx.detach().cpu().numpy())
        # one_hot_y.extend(np.eye(2)[true_label_vals])

        val_loss += loss.item()
        j = j+1
    
    val_loss = val_loss/j
    if(val_loss<min_val_loss):
        save_model(textcnn, name , val_loss)
        min_val_loss = val_loss

    matrix = confusion_matrix(true_labels, pred_labels)
    class_acc = matrix.diagonal()/matrix.sum(axis=1)
    print('Val loss: ', val_loss)    
    print('Val Accuracy: ',accuracy_score(true_labels,pred_labels))
    print('Val Macro F1 score: ',f1_score(true_labels,pred_labels,average='macro'))
    print('Val ROC AUC Score: ',roc_auc_score(true_labels , pred_logits))
    print(matrix)
    print('Class accuracies: ', class_acc)
    print('\n Time elapsed: ', time.time()-t0)
    scheduler.step()

======== Epoch 1 / 15 ========
Training loss:  0.9664912276847619
Accuracy:  0.07308704643730024
Macro F1 score:  0.07302457114110733
ROC_AUC Score:  0.09901656425948382
[[  5189   7068]
 [111258   4141]]
Class accuracies:  [0.42334992 0.03588419]
Test Accuracy:  0.8910501438553227
Test Macro F1 score:  0.6693913496670884
Test ROC AUC Score:  0.7758926850741007
[[83202  6447]
 [ 4156  3515]]
Class accuracies:  [0.9280862  0.45821927]
Val loss:  0.5648610510066421
Val Accuracy:  0.06523578254739151
Val Macro F1 score:  0.06443794453049465
Val ROC AUC Score:  0.04541378108711764
[[  575  2462]
 [27371  1507]]
Class accuracies:  [0.18933158 0.05218505]

 Time elapsed:  40.09598970413208
======== Epoch 2 / 15 ========
Training loss:  0.6610022707541187
Accuracy:  0.0516701134298427
Macro F1 score:  0.05160643834943955
ROC_AUC Score:  0.04007848925718592
[[  2775   9482]
 [111578   3821]]
Class accuracies:  [0.22640124 0.03311121]
Test Accuracy:  0.8987053020961776
Test Macro F1 score:  0.6

KeyboardInterrupt: ignored

In [0]:
load_model(textcnn, name)

Validation loss of the model is  0.5133204159433234


0.5133204159433234

In [0]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

true_labels = []
pred_labels = []
pred_logits = []
with torch.no_grad():
  for (x,y) in test_dataloader():
    pred_y = textcnn(x) # 64,2
    true_label_vals = y.detach().to('cpu').numpy()
    pred_label_vals = sigmoid(pred_y)
    true_labels.extend(true_label_vals)
    t = (pred_label_vals>=0.5).long()
    pred_labels += t.detach().cpu()
    pred_logits += pred_label_vals.detach().cpu()


In [0]:
accuracy = accuracy_score(true_labels,pred_labels)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(true_labels,pred_labels)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(true_labels,pred_labels)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(true_labels,pred_labels,average='macro')
print('F1 score: %f' % f1)
print("confusion matrix ", confusion_matrix(true_labels, pred_labels))

Accuracy: 0.900473
Precision: 0.393149
Recall: 0.483249
F1 score: 0.689505
confusion matrix  [[83927  5722]
 [ 3964  3707]]


In [0]:
# from sklearn.metrics import roc_curve
# from sklearn.metrics import roc_auc_score

# one_hot_y = np.eye(2)[test_y]
# pred_logits = [y.numpy() for y in pred_logits]
# auc = roc_auc_score(one_hot_y, pred_logits)
# # summarize scores
# print('Accuracy: ROC AUC=%.3f' % (auc))
# # calculate roc curves
# # ns_fpr, ns_tpr, _ = roc_curve(testy, ns_probs)
# fpr, tpr, _ = roc_curve(one_hot_y, pred_logits)
# # plot the roc curve for the model
# plt.plot(fpr, tpr, marker='.',label='roc curve')
# x1,x2,y1,y2 = plt.axis()
# plt.axis((x1,x2,0.8,1.0))
# # axis labels
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# # show the legend
# plt.legend()
# # show the plot
# plt.show()

In [0]:
test_nontoxic = test_df[test_df['toxic']==0]

In [0]:
test_nontoxic_texts = list(test_nontoxic['comment_text'].values)

In [0]:
# print(len(train_id_terms))
# count=0
# for word in train_id_terms:
#   if tokenizer.word_index.get(word) and tokenizer.word_index.get(word)<=5000:
#     print(word , tokenizer.word_index.get(word))
#     count+=1
# print(count)

In [0]:
# for word in test_id_terms:
#   print(word , tokenizer.word_index.get(word))

In [0]:
def CTF_gap(model,data,id_terms):
  # model - pytorch model
  # data - raw text -- list
  # list of id terms to replace. 
  model.eval()
  total_ctf_gap = 0
  total_inputs = 0
  dum='iamsurya'
  dumm='youarechan'
  count = 0
  print('Total sentences in dataset are: ', len(data)) 
  for i in range(len(data)):
    st = data[i]
    words = st.split()
    if(len(words)>10):
      continue  ## The paper says ctf is calculated only for those inputs that have less than 10 tokens.
    sentences = []
    sentences.append(data[i])
    
    ind_ctf = 0
    done = []
    for w in id_terms:
      if w in words:
        done.append(w)
        for wr in id_terms:
          if wr not in done:
            new = re.sub(r"\b%ss?\b" % w,dum,st)
            new = re.sub(r"\b%ss?\b" % wr,dumm,new)
            new = new.replace(dum,wr)
            new = new.replace(dumm,w)
            sentences.append(new)
    
    # No counterfactuals to consider
    if(len(sentences)<2):
      continue
    count += 1
    total_inputs += 1
    sent_sequences = tokenizer.texts_to_sequences(sentences)
    sent_data = pad_sequences(sent_sequences, maxlen=10, padding='post')
    # print(sent_data)

    sent_tensors = torch.tensor(sent_data,device=device,dtype=torch.long)
    with torch.no_grad():
      predictions = model(sent_tensors)
      predictions = sigmoid(predictions)
      # predictions = torch.max(predictions, dim=1)[1]

    for i in range(1,len(predictions)):
      ind_ctf += abs(predictions[0] - predictions[i])

    ind_ctf = ind_ctf.detach().cpu().numpy()
    ind_ctf = float(ind_ctf)/float(len(predictions)-1)
    total_ctf_gap += ind_ctf

  print('Total sentences used for ctf gap are: ', count)
  return total_ctf_gap/total_inputs


In [0]:
# sigmoid(torch.tensor([5.1]))

In [0]:
ctf = CTF_gap(textcnn, test_nontoxic_texts ,train_id_terms) # eval nt
print(ctf)

Total sentences in dataset are:  89649
Total sentences used for ctf gap are:  136
0.02285862796434543


In [0]:
ctf = CTF_gap(textcnn, test_nontoxic_texts ,test_id_terms) # for table 2
print(ctf)

Total sentences in dataset are:  89649
Total sentences used for ctf gap are:  52
0.12122252219162151


In [0]:
df = pd.read_csv(path+'synthetic_nontoxic.csv')
nontoxic_syn_texts = list(df['Text'].values)
ctf = CTF_gap(textcnn, nontoxic_syn_texts, train_id_terms)
print(ctf)

Total sentences in dataset are:  44436
Total sentences used for ctf gap are:  32007
0.006586309471391171


In [0]:
df = pd.read_csv(path+'synthetic_toxic.csv')
toxic_syn_texts = list(df['Text'].values)
ctf = CTF_gap(textcnn, toxic_syn_texts, train_id_terms)
print(ctf)

Total sentences in dataset are:  45047
Total sentences used for ctf gap are:  30578
0.014438790005869746


TPR,TNR gaps

In [0]:
id_tokens = train_id_terms + test_id_terms

In [0]:
dict_list={}
for x in id_tokens:
  dict_list[x]=[0,0,0,0] #tp,fp,fn,tn

In [0]:
batchsize = 1
def test_loader():
    for i in range(0, len(test_df),batchsize):
        id_dict={}
        count=0
        for x in id_tokens:
          if ((x in test_texts[i].split() or (len(x.split())==2) and x in test_texts[i])):
            id_dict[x]=1
            count=count+1
          else:
            id_dict[x]=0
        if (count==0):
          continue
        yield torch.tensor(test_data[i:i+batchsize], device=device, dtype=torch.long),torch.tensor(test_y[i:i+batchsize],device=device),id_dict

In [0]:
true_y = []
pred_y = []
t0 = time.time()
textcnn.eval()
total_loss = 0
total_steps = 0
pred_labels = []
true_labels = []
for batch in (test_loader()):
    total_steps+=1
    x,y,dct = batch
    lst = [k for k,v in dct.items() if v == 1]
    with torch.no_grad():
      fx = textcnn(x).squeeze(1)
    loss = criterion(fx,y.float())
    fx = sigmoid(fx)
    total_loss += loss.item()        
    true_label_vals = y.to('cpu').numpy()
    pred_label_vals = (fx>=0.5).long().detach().cpu().numpy()
    pred = pred_label_vals
    true = true_label_vals
    if(true==1 and pred==1):
      for x in lst:
        dict_list[x][0]+=1
    elif(true==1 and pred==0): # false positive
      for x in lst:
        dict_list[x][1]+=1
    elif(true==0 and pred==1): 
      for x in lst:
        dict_list[x][2]+=1
    elif(true==0 and pred==0):
      for x in lst:
        dict_list[x][3]+=1
    true_labels += list(true_label_vals.flatten())
    pred_labels += list(pred_label_vals)
avg_train_loss = total_loss / total_steps
print('avg_test_loss: ',avg_train_loss)
print('Accuracy: ',accuracy_score(true_labels,pred_labels))
true_y.extend(true_labels)
pred_y.extend(pred_labels)

avg_test_loss:  1.341417303014786
Accuracy:  0.8615069679552791


In [0]:
tpr={}
for x in dict_list:
  p=dict_list[x][0]+dict_list[x][2]
  if p==0:
    tpr[x]=0
  else:
    tpr[x]=dict_list[x][0]/p
tnr={}
for x in dict_list:
  n=dict_list[x][1]+dict_list[x][3]
  if n==0:
    tnr[x]=0
  else:
    tnr[x]=dict_list[x][3]/n

In [0]:
tpr_gap=0
tnr_gap=0
done=[]

for x in tpr:
  done.append(x)
  for y in tpr:
    if y not in done:
      tpr_gap+=abs(tpr[x]-tpr[y])
      tnr_gap+=abs(tnr[x]-tnr[y])


tpr_gap/=1225
tnr_gap/=1225
print('TNR gap: ',tnr_gap)
print('TPR gap: ',tpr_gap)


TNR gap:  0.15648930960634064
TPR gap:  0.26062330697187364
